In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
import sys
!{sys.executable} -m pip install beautifulsoup4

In [13]:
from bs4 import BeautifulSoup

In [14]:
import sys
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install html5lib
# !conda install lxml --yes
# !conda install html5lib --yes

# SCRAPING WEBSITE

In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_lst = pd.read_html(url)
type(df_lst)
print(len(df_lst))

list

3


In [16]:
df_lst1 = df_lst[0]
df_lst1[0:4]
len(df_lst1)
# df_lst1[1][1]
df_lst1_arr = np.array(df_lst1)
df_lst1_arr.shape
# df_lst1_arr[0][0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


180

(180, 3)

# CREATING DATAFRAME

In [17]:
#Creating data frame from the array
neighborhoods=pd.DataFrame(df_lst1_arr)
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods.columns = column_names 
#Selecting only rows where borough name is assigned
neighborhoods=neighborhoods[neighborhoods['Borough'] != 'Not assigned']
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


## Replacing any non assigned neighborhood to borough name

In [18]:

for i in range(0,neighborhoods.shape[0]):
        if neighborhoods.iloc[i,2] == 'Not assigned':
            neighborhoods.iloc[i,2] = neighborhoods.loc[i,0]
            

In [21]:
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [22]:
import sys
!{sys.executable} -m pip install geocoder


     |████████████████████████████████| 102kB 5.9MB/s ta 0:00:011
     |████████████████████████████████| 829kB 8.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [97]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('M5A'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

# Reading csv file, with longitudes and latittudes

In [23]:
#Create data frame with the Geospatial_Coordinates file
df1 = pd.read_csv('/resources/labs/DP0701EN/Geospatial_Coordinates.csv')
df1.head(10)
df1.shape[0]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


103

# Creating Dataframe with latitude and longitide columns

In [24]:
#Create columns for latitude and Longitude in neighborhood dataframe
neighborhoods['Latitude']= ""
neighborhoods['Longitude']= ""
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,,
3,M4A,North York,Victoria Village,,
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
5,M6A,North York,"Lawrence Manor, Lawrence Heights",,
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,


In [25]:
#Populate Latitude Longitude in neighborhood with df1
for i in range(0,neighborhoods.shape[0]):
    for j in range(0, df1.shape[0]):
        if neighborhoods.iloc[i,0] == df1.iloc[j,0]:
            neighborhoods.iloc[i,3] = df1.iloc[j,1]
            neighborhoods.iloc[i,4] = df1.iloc[j,2]    

In [26]:
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7533,-79.3297
3,M4A,North York,Victoria Village,43.7259,-79.3156
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
9,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
11,M3B,North York,Don Mills,43.7459,-79.3522
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [27]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [28]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [29]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="trnt_explorer")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


# create map of Toronto using latitude and longitude values for all neighborhoods

In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto


# Set up cliend credentials for Foursqaure API

In [31]:
CLIENT_ID = 'IJFLQV3IMIWD4YD52201FQZKKPXPVIS5P5M4K00ALNGY5TLE' # your Foursquare ID
CLIENT_SECRET = 'EMLUFXPWEFTPJU4KXN52RONCRSA4OX1SMIT3Q0O2TVB3X5IF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IJFLQV3IMIWD4YD52201FQZKKPXPVIS5P5M4K00ALNGY5TLE
CLIENT_SECRET:EMLUFXPWEFTPJU4KXN52RONCRSA4OX1SMIT3Q0O2TVB3X5IF


# Filtering Dataframe from Downtow Toronto

In [33]:
torontoDT_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
torontoDT_data.head(10)
torontoDT_data.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
5,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
6,M6G,Downtown Toronto,Christie,43.6695,-79.4226
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.6408,-79.3818
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6472,-79.3816


(19, 5)

In [34]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="TrntDT_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


# Crate map of downtown Toronto

In [36]:
# create map of DT Toronto using latitude and longitude values
map_DTtoronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to map
for lat, lng, borough, neighborhood in zip(torontoDT_data['Latitude'], torontoDT_data['Longitude'], torontoDT_data['Borough'], torontoDT_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DTtoronto) 
    
map_DTtoronto

# Create function for searching venues using Foursqaure API

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 25):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Create dataframe with venues

In [39]:
TorontoDT_venues = getNearbyVenues(names=torontoDT_data['Neighborhood'],
                                   latitudes=torontoDT_data['Latitude'],
                                   longitudes=torontoDT_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [40]:
TorontoDT_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
8,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
9,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center


In [41]:
TorontoDT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,25,25,25,25,25,25
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,25,25,25,25,25,25
Christie,16,16,16,16,16,16
Church and Wellesley,25,25,25,25,25,25
"Commerce Court, Victoria Hotel",25,25,25,25,25,25
"First Canadian Place, Underground city",25,25,25,25,25,25
"Garden District, Ryerson",25,25,25,25,25,25
"Harbourfront East, Union Station, Toronto Islands",25,25,25,25,25,25


In [42]:
print('There are {} uniques categories.'.format(len(TorontoDT_venues['Venue Category'].unique())))

There are 139 uniques categories.


# No of unique categories

In [44]:
TorontoDT_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa',
       'Restaurant', 'Park', 'Pub', 'Historic Site', 'Breakfast Spot',
       'Gym / Fitness Center', 'Farmers Market', 'Chocolate Shop',
       'Dessert Shop', 'Performing Arts Venue', 'French Restaurant',
       'Theater', 'Café', 'Mexican Restaurant', 'Portuguese Restaurant',
       'Creperie', 'Beer Bar', 'Sushi Restaurant', 'Yoga Studio',
       'Arts & Crafts Store', 'Burrito Place', 'Japanese Restaurant',
       'Hobby Shop', 'Diner', 'Fried Chicken Joint', 'Smoothie Shop',
       'Bank', 'Sandwich Place', 'Clothing Store', 'Comic Shop', 'Plaza',
       'Pizza Place', 'Music Venue', 'Burger Joint', 'Movie Theater',
       'Thai Restaurant', 'Electronics Store', 'Sporting Goods Shop',
       'College Rec Center', 'Ramen Restaurant', 'Shopping Mall',
       'Art Gallery', 'Bookstore', 'Italian Restaurant', 'Gym',
       'Food Truck', 'Middle Eastern Restaurant', 'Cosmetics Shop',
       'Gastropub', 'BBQ Joint', 'Americ

# One Hot Encoding

In [46]:
# one hot encoding
TorontoDT_onehot = pd.get_dummies(TorontoDT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TorontoDT_onehot['Neighborhood'] = TorontoDT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [TorontoDT_onehot.columns[-1]] + list(TorontoDT_onehot.columns[:-1])
TorontoDT_onehot = TorontoDT_onehot[fixed_columns]

# TorontoDT_onehot.head();
TorontoDT_grouped = TorontoDT_onehot.groupby('Neighborhood').mean().reset_index()
TorontoDT_grouped.head()
TorontoDT_grouped.shape

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Ethiopian Restaurant,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.04,0.00,0.0,0.0,0.0,0.0000,0.04,0.0,0.0000,0.04,0.04,0.0,0.04,0.0000,0.00,0.0000,0.04,0.00,0.00,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.04,0.0400,0.0,0.0,0.0,0.0,0.00,0.04,0.0,0.00,0.00,0.0,0.0,0.0000,0.0,0.0,0.00,0.08,0.0,0.0,0.0,0.04,0.04,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.00,0.0,0.0400,0.04,0.04,0.0,0.00,0.0,0.04,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.04,0.0,0.0,0.0000,0.0,0.040,0.0,0.0,0.00,0.0000,0.0,0.0,0.00,0.00,0.00,0.00,0.0000,0.0400,0.0,0.0,0.00,0.00,0.0000,0.08,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.04,0.0,0.04,0.0,0.00,0.0,0.0,0.04,0.0,0.0,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.125,0.125,0.125,0.0,0.00,0.00,0.0,0.0,0.0,0.0000,0.00,0.0,0.0625,0.00,0.00,0.0,0.00,0.0625,0.00,0.0625,0.00,0.00,0.00,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.0625,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.00,0.0,0.0000,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.00,0.0625,0.0,0.0,0.00,0.00,0.00,0.00,0.0625,0.0000,0.0,0.0,0.00,0.00,0.0625,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
2,Central Bay Street,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.00,0.04,0.0,0.0,0.0,0.0000,0.00,0.0,0.0400,0.00,0.00,0.0,0.00,0.0000,0.00,0.0000,0.00,0.04,0.00,0.0,0.0,0.0400,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.2000,0.0,0.0,0.0,0.0,0.04,0.00,0.0,0.00,0.00,0.0,0.0,0.0000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.04,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.00,0.0,0.0800,0.04,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.04,0.04,0.04,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.040,0.0,0.0,0.00,0.0000,0.0,0.0,0.04,0.04,0.00,0.00,0.0000,0.0000,0.0,0.0,0.00,0.04,0.0000,0.04,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.04,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.04,0.0,0.0,0.0
3,C

(19, 139)

# Sorting by top venues

In [47]:
num_top_venues = 5

# for hood in TorontoDT_grouped['Neighborhood']:
#     print("----"+hood+"----");
#     temp = TorontoDT_grouped[TorontoDT_grouped['Neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TorontoDT_grouped['Neighborhood']

for ind in np.arange(TorontoDT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TorontoDT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cocktail Bar,Park,Concert Hall,Liquor Store,Breakfast Spot,Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Bar,Rental Car Location,Boutique,Sculpture Garden,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Italian Restaurant,Spa,Modern European Restaurant,Park,Miscellaneous Shop,Middle Eastern Restaurant,Café,Poke Place,Portuguese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Baby Store,Restaurant,Italian Restaurant,Nightclub,Diner
4,Church and Wellesley,Gay Bar,Beer Bar,Coffee Shop,Creperie,Salon / Barbershop,Bookstore,Restaurant,Breakfast Spot,Ramen Restaurant,Pub
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Gastropub,Restaurant,Japanese Restaurant,Steakhouse,Museum,Gluten-free Restaurant,Pub,Deli / Bodega
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym / Fitness Center,Bakery,Gym,General Travel,Gastropub,Hotel,Gluten-free Restaurant
7,"Garden District, Ryerson",Café,Clothing Store,Music Venue,Sporting Goods Shop,Ramen Restaurant,Comic Shop,Sandwich Place,Movie Theater,Shopping Mall,Diner
8,"Harbourfront East, Union Station, Toronto Islands",Park,Café,Plaza,Performing Arts Venue,Ice Cream Shop,New American Restaurant,Pizza Place,Bubble Tea Shop,Lake,Roof Deck
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Wine Bar,Bar,Fish Market,Farmers Market,Dessert Shop,Organic Grocery


# K Nearest Neighbor CLustering and displaying of map

In [49]:
# set number of clusters
kclusters = 5

TorontoDT_grouped_clustering = TorontoDT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TorontoDT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TorontoDT_merged = torontoDT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TorontoDT_merged = TorontoDT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TorontoDT_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TorontoDT_merged['Latitude'], TorontoDT_merged['Longitude'], TorontoDT_merged['Neighborhood'], TorontoDT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

array([4, 2, 0, 3, 2, 4, 4, 2, 2, 4], dtype=int32)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0,Coffee Shop,Park,Breakfast Spot,Bakery,Gym / Fitness Center,Farmers Market,Performing Arts Venue,Café,Pub,Mexican Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,0,Coffee Shop,Diner,Japanese Restaurant,Park,Burrito Place,Portuguese Restaurant,Mexican Restaurant,Creperie,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,2,Café,Clothing Store,Music Venue,Sporting Goods Shop,Ramen Restaurant,Comic Shop,Sandwich Place,Movie Theater,Shopping Mall,Diner
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,4,Restaurant,Café,Gastropub,Coffee Shop,Food Truck,Diner,Creperie,Middle Eastern Restaurant,Farmers Market,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,4,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cocktail Bar,Park,Concert Hall,Liquor Store,Breakfast Spot,Restaurant


In [50]:
TorontoDT_merged.loc[TorontoDT_merged['Cluster Labels'] == 0, TorontoDT_merged.columns[[2] + list(range(5, TorontoDT_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Breakfast Spot,Bakery,Gym / Fitness Center,Farmers Market,Performing Arts Venue,Café,Pub,Mexican Restaurant
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Diner,Japanese Restaurant,Park,Burrito Place,Portuguese Restaurant,Mexican Restaurant,Creperie,Sandwich Place,Distribution Center
5,Central Bay Street,0,Coffee Shop,Italian Restaurant,Spa,Modern European Restaurant,Park,Miscellaneous Shop,Middle Eastern Restaurant,Café,Poke Place,Portuguese Restaurant


In [51]:
TorontoDT_merged.loc[TorontoDT_merged['Cluster Labels'] == 1, TorontoDT_merged.columns[[2] + list(range(5, TorontoDT_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,1,Park,Playground,Trail,Wine Bar,Comic Shop,Deli / Bodega,Dance Studio,Creperie,Cosmetics Shop,Concert Hall


In [52]:
TorontoDT_merged.loc[TorontoDT_merged['Cluster Labels'] == 2, TorontoDT_merged.columns[[2] + list(range(5, TorontoDT_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",2,Café,Clothing Store,Music Venue,Sporting Goods Shop,Ramen Restaurant,Comic Shop,Sandwich Place,Movie Theater,Shopping Mall,Diner
8,"Harbourfront East, Union Station, Toronto Islands",2,Park,Café,Plaza,Performing Arts Venue,Ice Cream Shop,New American Restaurant,Pizza Place,Bubble Tea Shop,Lake,Roof Deck
13,"CN Tower, King and Spadina, Railway Lands, Har...",2,Airport Lounge,Airport Service,Airport Terminal,Plane,Bar,Rental Car Location,Boutique,Sculpture Garden,Boat or Ferry,Coffee Shop
18,Church and Wellesley,2,Gay Bar,Beer Bar,Coffee Shop,Creperie,Salon / Barbershop,Bookstore,Restaurant,Breakfast Spot,Ramen Restaurant,Pub


In [53]:
TorontoDT_merged.loc[TorontoDT_merged['Cluster Labels'] == 3, TorontoDT_merged.columns[[2] + list(range(5, TorontoDT_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,3,Grocery Store,Café,Park,Candy Store,Coffee Shop,Baby Store,Restaurant,Italian Restaurant,Nightclub,Diner


In [54]:
TorontoDT_merged.loc[TorontoDT_merged['Cluster Labels'] == 4, TorontoDT_merged.columns[[2] + list(range(5, TorontoDT_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,4,Restaurant,Café,Gastropub,Coffee Shop,Food Truck,Diner,Creperie,Middle Eastern Restaurant,Farmers Market,Cosmetics Shop
4,Berczy Park,4,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cocktail Bar,Park,Concert Hall,Liquor Store,Breakfast Spot,Restaurant
7,"Richmond, Adelaide, King",4,Coffee Shop,Hotel,Steakhouse,Gym / Fitness Center,Speakeasy,Café,General Travel,Pizza Place,Plaza,Concert Hall
9,"Toronto Dominion Centre, Design Exchange",4,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym / Fitness Center,Steakhouse,Pizza Place,Concert Hall,Pub,Deli / Bodega
10,"Commerce Court, Victoria Hotel",4,Café,Coffee Shop,Gastropub,Restaurant,Japanese Restaurant,Steakhouse,Museum,Gluten-free Restaurant,Pub,Deli / Bodega
11,"University of Toronto, Harbord",4,Café,Restaurant,Bookstore,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,College Gym,Chinese Restaurant,Comfort Food Restaurant
12,"Kensington Market, Chinatown, Grange Park",4,Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Wine Bar,Bar,Fish Market,Farmers Market,Dessert Shop,Organic Grocery
15,Stn A PO Boxes,4,Beer Bar,Seafood Restaurant,Cocktail Bar,Café,Farmers Market,Food Truck,Jazz Club,Japanese Restaurant,Concert Hall,Hotel
16,"St. James Town, Cabbagetown",4,Café,Italian Restaurant,Restaurant,Pet Store,Market,Butcher,Caribbean Restaurant,Park,Coffee Shop,Diner
17,"First Canadian Place, Underground city",4,Café,Coffee Shop,Restaurant,Gym / Fitness Center,Bakery,Gym,General Travel,Gastropub,Hotel,Gluten-free Restaurant
